In [ ]:
!pip install -U scikit-learn
!pip install pandas
!pip install numpy
!pip install matplotlib

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Initially used numpy for reading csv but it gave error at line 38 because there was a comma. So i guess it is not for reading text. therefore used pandas.

In [36]:
# train data
tdf = pd.read_csv("CompleteDataset.csv", sep="|")
# test data
#vdf = pd.read_csv('defects.csv')
cdf = vdf.sample(frac = 1, random_state=1).reset_index()
#print(tdf.head)

vdf = tdf.iloc[30400:]
tdf = tdf.iloc[:30400]

print(tdf.head)
print(vdf.head)

# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2


<bound method NDFrame.head of                                                  summary       category  \
0      Incorrect data format received from the third-...     Data Error   
1      System crashed due to unexpected null value in...     Data Error   
2      Unexpected datatype in the 'userAge' field cau...     Data Error   
3      The system could not process received XML data...     Data Error   
4      Unable to parse the incoming JSON data due to ...     Data Error   
...                                                  ...            ...   
30395  The test script relied on outdated APIs,result...  Testing Error   
30396  The test script did not handle session timeout...  Testing Error   
30397  The test execution was affected by network con...  Testing Error   
30398  The test script encountered issues with handli...  Testing Error   
30399  The test execution failed due to an issue with...  Testing Error   

                       super,,,,,  
0      Data Management Error,,,,,

Get number of samples and words per sample to compare. Then using https://developers.google.com/machine-learning/guides/text-classification/step-2-5 decide to the model.

In [ ]:
rows = len(df)
total_words = df['summary'].apply(lambda x: len(str(x).split())).sum()
w_per_s = total_words / rows
w_per_s

10.276476471207495

Since it is less than 1500 we are going with MLP.

In [ ]:
def ngram_vectorize(train_texts, train_labels, val_texts):
    """Vectorizes texts as n-gram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

    # Arguments
        train_texts: list, training text strings.
        train_labels: np.ndarray, training labels.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val: vectorized training and validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

Berfore we fed model with data. We should tokenize.

In [ ]:
# Define train_texts, train_labels, val_texts, and val_labels earlier
train_texts = tdf['summary']
train_labels = tdf['super']
val_texts = vdf['summary']
val_labels = vdf['super']
# Initialize a label encoder
label_encoder = LabelEncoder()

# Fit the encoder on the training labels and transform both train and validation labels

y_train_encoded = label_encoder.fit_transform(train_labels)
y_val_encoded = label_encoder.transform(val_labels)
unique_classes = label_encoder.classes_

print(unique_classes)

# Vectorize the texts using the function
x_train, x_val = ngram_vectorize(train_texts, train_labels, val_texts)

# Initialize the MLP classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=100, activation='relu', solver='adam', random_state=42)

# Train the model
mlp_clf.fit(x_train, y_train_encoded)

# Predict on the validation set
y_val_pred_mlp = mlp_clf.predict(x_val)

# Calculate the accuracy
accuracy_mlp = accuracy_score(y_val_encoded, y_val_pred_mlp, normalize=True)
print(accuracy_mlp)
cm = confusion_matrix(y_val_encoded, y_val_pred_mlp)
fig = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=unique_classes)
fig.plot()
plt.show()

print(classification_report(y_val_encoded, y_val_pred_mlp))

KeyError: ignored